In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import alpaca_trade_api as tradeapi
import logging
import math

logging.basicConfig(filename='trading_bot.log', level=logging.INFO, format='%(asctime)s %(message)s')


In [ ]:
# Load data
df = pd.read_csv('sp500_index.csv')
df['S&P500'] = df['S&P500'].astype(float)

# Preprocess data
scaler = MinMaxScaler(feature_range=(-1, 1))
dataset = scaler.fit_transform(df['S&P500'].values.reshape(-1, 1))

# Split into train and test sets
train_size = int(len(dataset) * 0.7)
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]


In [ ]:
# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Set the look back window
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape the inputs from [samples, time steps] into [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile and fit the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(trainX, trainY, epochs=20, batch_size=32)


In [ ]:
# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert predictions to get the actual predicted prices
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)


In [ ]:
# Set up Alpaca API
api_key = 'your_api_key'
api_secret = 'your_api_secret'
base_url = 'https://paper-api.alpaca.markets'
api = tradeapi.REST(api_key, api_secret, base_url)


In [ ]:
# Set initial variables
capital = 6000  # Initial capital
shares = 0  # Initially no shares
price = None  # Initially no buying price
bot_active = True  # Initially bot is active


In [ ]:
# Trading logic
for i in range(look_back, len(df)-1):
    if not bot_active:
        break

    # Prepare the data
    X = df['S&P500'].values[i-look_back:i].reshape(1, -1)
    current_price = df['S&P500'].values[i]
    next_day_price = df['S&P500'].values[i+1]

    # Use the model to make a prediction
    prediction = model.predict(X)

    # Decide whether to buy or sell
    if prediction > current_price and capital >= current_price:
        shares = math.floor(capital / current_price)  # Buy as many shares as possible
        try:
            api.submit_order(symbol='SPY', qty=shares, side='buy', type='market', time_in_force='gtc')
            price = current_price
            capital -= shares * current_price  # Update capital
            logging.info(f'Buying SPY at {current_price}, shares bought: {shares}')
        except tradeapi.rest.APIError as e:
            logging.error(f'Failed to submit the buy order: {str(e)}')
    elif prediction < current_price and shares > 0:
        try:
            api.submit_order(symbol='SPY', qty=shares, side='sell', type='market', time_in_force='gtc')
            capital += shares * current_price  # Update capital
            logging.info(f'Selling SPY at {current_price}, shares sold: {shares}')
            shares = 0  # Reset shares
            price = None  # Reset buying price
        except tradeapi.rest.APIError as e:
            logging.error(f'Failed to submit the sell order: {str(e)}')
